
# 01 — Análisis de Dinámicas Moleculares y Selección de Estructuras Representativas

**Propósito:** analizar trayectorias de dinámica molecular, extraer features, reducir dimensión, clusterizar, obtener centroides (frames representativos) y exportar PDBs 



# Instalación con conda en python 3.12
https://conda.io/miniconda.html

* anaconda3/bin/conda install biopython -c conda-forge
* anaconda3/bin/conda install numpy pandas matplotlib seaborn scikit-learn -c conda-forge
* anaconda3/bin/conda install tqdm ipykernel jupyterlab -c conda-forge
* anaconda3/bin/conda install mdtraj -c conda-forge
* anaconda3/bin/conda install mdanalysis -c conda-forge
* anaconda3/bin/conda install MDAnalysisTests -c conda-forge
* anaconda3/bin/conda install ambertools -c conda-forge
* anaconda3/bin/conda install nglview -c conda-forge
* anaconda3/bin/conda install prody -c conda-forge
* anaconda3/bin/conda install hdbscan -c conda-forge
* anaconda3/bin/conda install scikit-learn -c conda-forge
* anaconda3/bin/conda install umap-learn -c conda-forge
* anaconda3/bin/conda install plotly -c conda-forge


## 0. Configuración inicial

- Cargar librerías y funciones auxiliares.
- Definir rutas de entrada/salida y semilla aleatoria.


In [ ]:
def get_distances(traj):
    """Create a distance matrix"""
    import mdtraj as md
    from mdtraj.geometry.alignment import compute_average_structure
    import numpy as np
    traj_aligned = traj.superpose(traj[0], parallel=True)

    # matrix initialization
    distances = np.empty((traj.n_frames, traj.n_frames))
    # Pairwise RMSD calculation (matrix n²)
    for i in range(traj.n_frames):
        distances[i]=md.rmsd(traj_aligned,traj_aligned, frame=i)
    return distances

In [ ]:
def TSNE_cluster(distances, min_cluster_size):
    """Create a TSNE cluster"""
    from sklearn.manifold import TSNE
    import hdbscan
    import matplotlib.pyplot as plt

    model = TSNE(n_components=2,verbose=2,perplexity=10,learning_rate=100)
    data_tsne = model.fit_transform(distances)
    #plt.scatter(data_tsne.T[0],data_tsne.T[1],cmap="gnuplot")
    cluster = hdbscan.HDBSCAN(min_cluster_size)
    cluster.fit(data_tsne)
    plt.scatter(data_tsne.T[0],data_tsne.T[1],cmap="gnuplot",c=cluster.labels_)
    print(len(set(cluster.labels_)), set(cluster.labels_))
    return  data_tsne, cluster


## 1. Carga y preprocesamiento de trayectorias

- Cargar topología y trayectoria (MDAnalysis / MDTraj / pytraj).
- Alinear backbones, corregir PBC y aplicar stride.
- Guardar una copia reducida si procede para acelerar análisis.


In [ ]:
import mdtraj as md

topology_file = '/Users/ruta/archivo.top'  
trajectory_file = '/Users/ruta/archivo.nc'
traj = md.load(trajectory_file, top=topology_file)

In [ ]:
# en esta parte es posible que se deba corregir el "chainid 0 and resid 0 to 15" y ajustar al largo del peptido para cada saco 

residues = traj.topology.select('(chainid 0 and resid 0 to 15) and not resname ACE and not resname NME and not resname T4P and not resname CL and not resname NA')
traj_protein_only = traj.atom_slice(residues)
print (traj_protein_only)

In [ ]:
 for chain in traj_protein_only.topology.chains:
    print(f"Chain {chain.index}:")
    for residue in chain.residues:
        print(f"{residue.index} {residue.name}")

In [ ]:
sequence = []
for chain in traj_protein_only.topology.chains:
    for residue in chain.residues:
        sequence.append(residue.name)
sequence = "-".join(sequence)  # Unir los nombres de los residuos en una cadena



## 2. Selección de características (features)

- Decide qué features usar: distancias inter-residuo, ángulos, dihedrales, RMSD contra referencia.
- Guarda las features en disco (`.npz` o `.npy`) para no recalcular.


In [ ]:
traj = traj_protein_only.superpose(traj_protein_only[0], parallel=True)
print("Trajectory superpose!")
distances_traj = get_distances(traj)
print("Distances matrix ready!")

In [ ]:
rmsds_traj = md.rmsd(traj_protein_only, traj_protein_only, 0)
print("RMSD done!")
data_tsne_traj,cluster_traj   = TSNE_cluster(distances_traj, 20) #este ultimo valor se puede ajustar, este indica el valor minimo de estructuras necesarias para crear un cluster 
print("Clustering ready!")

In [ ]:
# creacion de un data frame 

import pandas as pd
import numpy as np

reference = traj[0]
rmsd = md.rmsd(traj, reference)

rmsd_angstrom = rmsd 

data_rmsds_Gn = pd.DataFrame([rmsd], index=["RMSD"])
data_rmsds_Gn = data_rmsds_Gn.T
data_rmsds_Gn["time"] = np.arange(len(data_rmsds_Gn ["RMSD"]))

#data_rmsds_Gn['y_cluster'] = np.full((1, 1002), -0.1)[0]
#data_rmsds_Gn['x_cluster'] = cluster_traj.labels_


## 3. Reducción dimensional

- Utiliza PCA / tICA / UMAP / t-SNE según necesidad.
- Guarda embeddings para su reutilización.


In [ ]:

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

def plot_rmsd_with_average(data_rmsds_Gn, cluster_traj):
    # Calcular el valor promedio del RMSD
    average_rmsd = data_rmsds_Gn['RMSD'].mean()
    
    # Configuración del gráfico
    plt.figure(1, figsize=(20, 6))
    sns.set(style="whitegrid")
    sns.set(style="white")
    
    plot = sns.lineplot(x="time", y="RMSD", data=data_rmsds_Gn, palette="tab10", linewidth=1.5, dashes=False)
    
    # Puntos de colores según los clusters
    plt.scatter(data_rmsds_Gn['time'], np.full((1, len(data_rmsds_Gn)), -0.1), cmap="gnuplot", c=cluster_traj.labels_[:])
    
    # Añadir el valor promedio del RMSD al gráfico
    plt.axhline(y=average_rmsd, color='r', linestyle='--', label=f'Average RMSD: {average_rmsd:.2f} nm')
    
    plt.title('RMSD')
    plt.xlabel("time(ns)")
    plt.ylabel(r"RMSD (nm)")
    plt.legend(loc='upper left')
    
    plt.show()
    
    return average_rmsd

average_rmsd = plot_rmsd_with_average(data_rmsds_Gn, cluster_traj)
#print(f"El valor promedio del RMSD es: {average_rmsd:.2f} nm")


## 4. Clustering

- Ejecuta HDBSCAN (recomendado) o KMeans.
- Guarda etiquetas y parámetros del clustering.
- Evalúa con métricas (tamaño de cluster, silhouette, estabilidad).


In [ ]:
from collections import Counter
import pandas as pd
import seaborn as sns

# Generar los datos del gráfico
labels, values = zip(*Counter(cluster_traj.labels_).items())
data8 = pd.DataFrame([labels, values], ["Groups", "Frames"])
data8 = data8.T

# Identificar el grupo con el mayor valor de Frames
largest_group = data8.loc[data8["Frames"].idxmax()]
group_max = int(largest_group["Groups"])
frame_max = int(largest_group["Frames"])

# Guardar el resultado en una variable
frem = frame_max

# Visualizar el gráfico
ax = sns.barplot(x="Groups", y="Frames", data=data8[:], palette="gnuplot")




## 5. Cálculo de centroides

- En feature space o embedding space, calcula el centro de cada cluster.
- Encuentra el frame más cercano a ese centro y extráelo como PDB.
- Exporta PDBs en `results/md_analysis/centroids/`.


In [ ]:
def get_centroid(traj, cluster, name, select=-1):
    """Centroid"""
    traj_aligned = traj.superpose(traj[0], parallel=True)
    
    from collections import defaultdict
    tally = defaultdict(list)
    
    for i,item in enumerate(cluster.labels_):
        tally[item].append(i)
    labels = list(set(cluster.labels_))
    frames = []
    
    if select < 0:
        labels = labels[:-1]
    if select > 0:
        labels = [labels[select]]
        
    for lab in labels:
        traj_cluster = traj_aligned.slice(tally[lab])
        atom_indices = [a.index for a in traj_cluster.topology.atoms if a.element.symbol != 'H']
        distances = np.empty((traj_cluster.n_frames, traj_cluster.n_frames))
        for i in range(traj_cluster.n_frames):
            distances[i] = md.rmsd(traj_cluster, traj_cluster, i, atom_indices=atom_indices)
        beta = 1
        index = np.exp(-beta*distances / distances.std()).sum(axis=1).argmax()
        print(index)
        frames.append((traj_cluster.n_frames, index))
        centroid = traj_cluster[index]
        centroid.save_pdb("%s_c%s_%03d.pdb"%(name, lab, index))
        
        frames = []
        for i,value in enumerate(cluster_traj.labels_):
            if value == lab:
                frames.append(i)
        global_index = frames[index]

    return index, global_index, centroid

In [ ]:
# Obtener el centroide utilizando el grupo identificado
index, global_index, centroid = get_centroid(traj, cluster_traj, "traj", 0)


In [ ]:
# Identificar el frame representativo del grupo más grande
frames = []
for i, value in enumerate(cluster_traj.labels_):
    if value == 0:  # Usar el grupo más grande
        frames.append(i)
len(frames)
frames[213]


## 6. Visualización avanzada

- Reproducir las estructuras representativas (nglview, py3Dmol).
- Guardar figuras que ilustren clusters y estructuras.


In [ ]:
frame_index = 158 # en esta parte se debe coloar el valor del frame más representativo del centroide
frame = traj[frame_index]

# Guardar el frame seleccionado como archivo PDB
output_pdb_file = f'estructura_frame_{frame_index}.pdb'
frame.save_pdb(output_pdb_file)

print(f"Estructura 3mgn del frame {frame_index} guardada en {output_pdb_file}")


## 7. Exportación de resultados

- Tabla resumen: `results/md_analysis/summary_clusters.csv` (cluster_id, n_frames, centroid_frame, notas).
- Guardar todo lo necesario para el notebook 02 (PDBs de centroides, json con índices, embeddings).


In [ ]:
# Extraer el nombre del archivo de topología (1t7d)
name = topology_file.split('/')[-1].split('.')[0]  # Extraer "1t7d" del nombre del archivo

frames = traj_protein_only.n_frames
residues_count = traj_protein_only.n_residues
atoms_count = traj_protein_only.n_atoms

# Calcular el número de conformaciones encontradas (ignorando el grupo -1)
labels, values = zip(*Counter(cluster_traj.labels_).items())
data8 = pd.DataFrame([labels, values], ["Groups", "Frames"]).T
data8 = data8[data8["Groups"] != -1]  # Ignorar el grupo -1
num_conformations = len(data8)

# Obtener el grupo más grande
largest_group = data8.loc[data8["Frames"].idxmax()]
group_max = int(largest_group["Groups"])

# Obtener el frame representativo del grupo más grande
_, representative_frame, _ = get_centroid(traj_protein_only, cluster_traj, name, select=group_max)

# Crear la tabla resumen
summary_table = pd.DataFrame({
    "Name": [name],
    "Frames": [frames],
    "Residues": [residues_count],
    "Atoms": [atoms_count],
    "Sequence": [sequence],
    "RMSD (Å)": [average_rmsd],
    "Conformaciones": [num_conformations],
    "Frame Representativo": [representative_frame]
})

# Aplicar formato a la tabla
styled_table = summary_table.style \
    .format(precision=3, thousands=".", decimal=",") \
    .format_index(str.upper, axis=1) \
    .set_properties(**{'text-align': 'left'}) \
    .set_table_styles([{
        'selector': 'th',
        'props': [('background-color', '#4CAF50'), ('color', 'white')]
    }])

# Mostrar la tabla con estilo
styled_table

summary_table = pd.DataFrame({
    "Name": [name],
    "Frames": [frames],
    "Residues": [residues_count],
    "Atoms": [atoms_count],
    "Sequence": [sequence],
    "RMSD (Å)": [average_rmsd]
})

# Formato a la tabla
styled_table = summary_table.style \
    .format(precision=3, thousands=".", decimal=",") \
    .format_index(str.upper, axis=1) \
    .set_properties(**{'text-align': 'left'}) \
    .set_table_styles([{
        'selector': 'th',
        'props': [('background-color', '#4CAF50'), ('color', 'white')]
    }])

styled_table

# Guardar la tabla 
# summary_table.to_csv(f"{name}_summary_table.csv", index=False)